In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00


In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00


## Outline

- Interact with OpenAI via direct API calls
- Undestanding API calls through LangChain:
    - Models
        - LLMs
        - Chat Models
            - Schema
    - Prompts
    - Chain
    - Memory
    - Output parsers

To begin, import the necessary dependencies into your project and access the environment file to retrieve the OpenAPI key.

In [ ]:
import os
import openai
os.environ["OPENAI_API_KEY"] = "sk-Th0ZmtEfXJB9RperrVhpT3BlbkFJzbc1wFVVa**************"

LangChain provides interfaces and integrations for two types of models:

1. LLMs: Models that take a text string as input and return a text string.

2. Chat models: Models that are backed by a language model but take a list of Chat Messages as input and return a Chat Message.


Establish a connection with a language model using LangChain

In [ ]:
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI()

In [ ]:
llm.temperature

0.7

The LLM **<span style="background-color: yellow">temperature</span>** serves as a hyperparameter controlling the level of **<span style="background-color: yellow">randomness</span>**  or creativity in the AI's responses. A higher temperature value generally leads to more diverse and imaginative outputs, but it can also increase the chances of the AI deviating from the intended context.

In [ ]:
llm.model_name

'text-davinci-003'

In [ ]:
llm("tell me something I do not know")

'\n\nThe average human will spend six months of their life waiting at red lights.'

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI()

In [ ]:
chat.temperature

0.7

In [ ]:
chat.model_name

'gpt-3.5-turbo'

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

`AIMessage`  
A chat message representing information coming from the AI system.

`HumanMessage`  
A chat message representing information coming from a human interacting with the AI system.

`SystemMessage`  
A chat message representing information that should be instructions to the AI system.

In [ ]:
chat_model = ChatOpenAI(temperature=0)
message=chat_model.predict_messages([HumanMessage(content="Can you translate the following sentence into Hindi for me: 'I love nature.")])
message

AIMessage(content='मुझे प्रकृति पसंद है। (Mujhe prakriti pasand hai.)', additional_kwargs={}, example=False)

Let's us try to interact with OpenAPI's Chat API using Langchain

We will now define a template:

In [ ]:
template_string = """Fill in the details \
that are required for {purpose}. \
Include the following information: {info}.
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [ ]:
customer_style = """Indian English \
in a calm and respectful tone
"""

In [ ]:
customer_email = """
I'm interested in purchasing a Honda Civic 2022 and would like to know if you have it in stock. \
Availability: Is the Honda Civic 2022 currently available? \
Price: What is the price for the Honda Civic 2022? \
Options: Are there any additional options or packages available? \
"""

In [ ]:
customer_messages = prompt_template.format_messages(
                    info=customer_style,
                    purpose=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [ ]:
print(customer_messages[0])

In [ ]:
customer_response = chat(customer_messages)

In [ ]:
print(customer_response.content)

In [ ]:
sales_reply = """Thank you for your inquiry regarding the Honda Civic 2022 \
Availability: Yes, the Honda Civic 2022 is in stock. \
Price: The price for the Honda Civic 2022 starts at ₹22,50,000. \
Options: We offer various customization options and packages for the Honda Civic 2022 to suit your preferences. \
please feel free to reach out to our sales team at 9234567890 or visit our dealership. \
"""

In [ ]:
sales_style = """\
a polite tone \
that speaks in steve harvey style \
"""

In [ ]:
sales_messages = prompt_template.format_messages(
    info=sales_style,
    purpose=sales_reply)

print(sales_messages[0].content)

In [ ]:
sales_response = chat(sales_messages)

In [ ]:
print(sales_response.content)

### Let's establish the desired format for the output generated by the Language Model:

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
# Define the data model for a car sale
class CarSale(BaseModel):
    model: str = Field(description="name of the car model")
    year: int = Field(description="year of manufacture")
    price: float = Field(description="price in INR")
    color: str = Field(description="color of the car")

In [ ]:
# Define the query for generating a car sale
car_sale_query = "Generate a JSON object for a honda city car sale."

# Set up the parser and the prompt template
parser = PydanticOutputParser(pydantic_object=CarSale)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
# Choose an LLM and a temperature
model_name = "text-davinci-003"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature)

In [ ]:
# Generate the output and parse it
_input = prompt.format_prompt(query=car_sale_query)
output = model(_input.to_string())
parser.parse(output)

# Memory
    - ConversationBufferMemory
    - ConversationBufferWindowMemory
    - ConversationSummaryMemory

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryBufferMemory

In [ ]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
#     verbose=True
)

In [ ]:
conversation.predict(input="Hi, How are you today? ")

In [ ]:
conversation.predict(input="I'm Vaibhav and taking a workshop on cooking clases in indian cusines")

In [ ]:
conversation.predict(input="I'm can you give few options in 20 words")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
memory = ConversationBufferWindowMemory(k=1)

In [ ]:
conversation.predict(input="whats my name")

In [ ]:
recipe = "Preheat oven to 375 degrees F. \
In a large bowl, toss apples with lemon juice and vanilla. \
Mix in 1/4 cup of sugar, 1/4 cup of flour, cinnamon, nutmeg and salt. \
Pour apple mixture into a 9x13 inch baking dish. \
In another bowl, combine oats, brown sugar, butter and remaining flour. \
Sprinkle over apple mixture. \
Bake for 25 to 30 minutes or until apples are tender and topping is golden brown."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hi, I'm Vaibhav"}, {"output": "Hello, I'm Pooja"})
memory.save_context({"input": "Do you like apple crisp?"},
                    {"output": "Yes, I love it"})
memory.save_context({"input": "Do you know how to make it?"},
                    {"output": f"{recipe}"})

In [ ]:
memory.load_memory_variables({})

# Chain

- LLMChain
- Sequential Chains
    - SimpleSequentialChain
    - SequentialChain
- Router Chain


In [ ]:
import pandas as pd
df = pd.read_csv('./llmsdemo/Leads.csv')

In [ ]:
df.head()

In [ ]:
column_name = df.columns.tolist()

In [ ]:
column_name

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the most common lead source?" \
    "lead profile contain{query}"

)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
query = "mumbai"
chain.run(query)

In [ ]:
from langchain.chains import SimpleSequentialChain

SimpleSequentialChain

# First Chain

In [ ]:
first_prompt = ChatPromptTemplate.from_template( "How many {column_name} values are missing in the dataset?")
chain_first = LLMChain(llm=llm, prompt=first_prompt)

# Second Chain

In [ ]:
second_prompt = ChatPromptTemplate.from_template( "What is the distribution of {column_name} values in the dataset?")
chain_second = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_first, chain_second],
                                             verbose=True)

In [ ]:
overall_simple_chain.run(column_name)

### Interacting with your own data locally

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.indexes import VectorstoreIndexCreator
import langchain

In [ ]:
loader = CSVLoader(file_path='./llmsdemo/leads_small.csv')

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
index

In [ ]:
query = "what are the unique leads origins in the dataset"

In [ ]:
response = index.query(query)

In [ ]:
display(Markdown(response))

### Evaluation

In [ ]:
from langchain.evaluation.qa import QAGenerateChain

In [ ]:
file = './llmsdemo/leads_small.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [ ]:
loader

In [ ]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [ ]:
llm = ChatOpenAI(temperature = 0.0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [ ]:
data[0]

Document(page_content='\ufeffProspect ID: 7927b2df-8bba-4d29-b9a2-b6e0beafe620\nLead Number: 660737\nLead Origin: API\nLead Source: Olark Chat\nDo Not Email: No\nDo Not Call: No\nConverted: 0\nTotalVisits: 0\nTotal Time Spent on Website: 0\nPage Views Per Visit: 0\nLast Activity: Page Visited on Website\nCountry: \nSpecialization: Select\nHow did you hear about X Education: Select\nWhat is your current occupation: Unemployed\nWhat matters most to you in choosing a course: Better Career Prospects\nSearch: No\nMagazine: No\nNewspaper Article: No\nX Education Forums: No\nNewspaper: No\nDigital Advertisement: No\nThrough Recommendations: No\nReceive More Updates About Our Courses: No\nTags: Interested in other courses\nLead Quality: Low in Relevance\nUpdate me on Supply Chain Content: No\nGet updates on DM Content: No\nLead Profile: Select\nCity: Select\nAsymmetrique Activity Index: 02.Medium\nAsymmetrique Profile Index: 02.Medium\nAsymmetrique Activity Score: 15\nAsymmetrique Profile Scor

In [ ]:
example_gen_chain = QAGenerateChain.from_llm(ChatOpenAI())

In [ ]:
new_examples = example_gen_chain.apply_and_parse(
    [{"doc": t} for t in data[:5]]
)

In [ ]:
new_examples[0]

{'query': 'What is the Lead Number mentioned in the document?',
 'answer': 'The Lead Number mentioned in the document is 660737.'}

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
llm = ChatOpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [ ]:
langchain.debug = True

In [ ]:
predictions = qa.apply(new_examples)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the Lead Number mentioned in the document?",
  "answer": "The Lead Number mentioned in the document is 660737."
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "What is the Lead Number mentioned in the document?",
  "context": "﻿Prospect ID: 88867067-3750-4753-8d33-1c7d1db53b5e\nLead Number: 660558\nLead Origin: Landing Page Submission\nLead Source: Organic Search\nDo Not Email: No\nDo Not Call: No\nConverted: 0\nTotalVisits: 5\nTotal Time Spent on Website: 170\nPage Views Per Visit: 5\nLast Activity: Email Opened\nCountry: India\nSpecialization: Business Administration\nHow did you hear about X Education: Select\nWhat is your current occupation: Unemployed\nWhat matters most to you in choosing a course: Bette

In [ ]:
graded_outputs = eval_chain.evaluate(new_examples, predictions)

In [ ]:
graded_outputs

[{'text': 'INCORRECT'},
 {'text': 'INCORRECT'},
 {'text': 'INCORRECT'},
 {'text': 'INCORRECT'},
 {'text': 'INCORRECT'}]

### Agents

## Search Engine Demo

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
search.run("Obama's first name?")

### Search for research paper in arxiv

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.0)
tools = load_tools(
    ["arxiv"],
)



> Entering new  chain...
I should use the arxiv tool to search for the paper with the given identifier.
Action: arxiv
Action Input: 1605.08386
Observation: Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on
fibers of a fixed integer matrix can be bounded from above by a constant. We
then study the mixing behaviour of heat-bath random walks on these graphs. We
also state explicit conditions on the set of moves so that the heat-bath random
walk, a generalization of the Glauber dynamics, is an expander in fixed
dimension.
Thought:The paper "1605.08386" is titled "Heat-bath random walks with Markov bases" and was published on May 26, 2016. It is authored by Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points with edges comin

'The paper "1605.08386" is about heat-bath random walks with Markov bases.'

In [ ]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent_chain.run(
    "What's the paper 1605.08386 about?",
)

### Interact like human

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0.0)
math_llm = OpenAI(temperature=0.0)
tools = load_tools(
    ["human", "llm-math"],
    llm=math_llm,
)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


agent_chain.run("What's my friend Eric's surname?")



> Entering new  chain...
I don't have access to personal information about individuals unless it has been shared with me in the course of this conversation. I should ask the human for guidance.
Action: human
Action Input: What is Eric's surname?

What is Eric's surname?
Zhu

Observation: Zhu
Thought:I now know Eric's surname.
Final Answer: Eric's surname is Zhu.

> Finished chain.


"Eric's surname is Zhu."

### search for youtube video

In [ ]:
from langchain.tools import YouTubeSearchTool

tool = YouTubeSearchTool()

tool.run("lex friedman,10")

"['/watch?v=Ff4fRgnuFgQ&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=-hxeDjAxvJ8&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=8qgIFD0PaWo&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=pdJQ8iVTwj8&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=L_Guz73e6fw&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=YDjOS0VHEr4&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=IQnUa5Bq5x4&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=PdE-waSx-d8&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=1Wy-6z17up4&pp=ygUMbGV4IGZyaWVkbWFu', '/watch?v=wMavKrA-4do&pp=ygUMbGV4IGZyaWVkbWFu']"